In [ ]:
import os
import requests
import pandas as pd
from sqlalchemy import create_engine
from tqdm import tqdm
from dotenv import load_dotenv
import aiohttp
import asyncio
from ratelimit import limits, sleep_and_retry

# Load environment variables
load_dotenv()
api_key = os.getenv('api')

# Database connection
engine = create_engine('mssql+pyodbc://Revision-PC\\SQLEXPRESS/RiotDataDB?driver=ODBC+Driver+17+for+SQL+Server')

# Rate limit parameters
RATE_LIMIT = 100  # 100 requests per minute
TIME_WINDOW = 60  # 60 seconds

# Decorator to handle rate limiting
@sleep_and_retry
@limits(calls=RATE_LIMIT, period=TIME_WINDOW)
async def fetch_match_detail(session, match_id):
    api_match_details = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}"
    try:
        async with session.get(api_match_details) as response:
            response.raise_for_status()
            details_json = await response.json()
            participants = details_json.get('info', {}).get('participants', [])
            participants_df = pd.json_normalize(participants)
            participants_df.insert(0, 'Match_ID', match_id)
            return participants_df
    except requests.exceptions.HTTPError as e:
        print(f"HTTPError for match_id {match_id}: {e}")
        return None

async def fetch_all_match_details(match_ids):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_match_detail(session, match_id) for match_id in match_ids]
        results = await asyncio.gather(*tasks)
    return [result for result in results if result is not None]

def main():
    # Read match IDs from the database
    match_df = pd.read_sql_query("SELECT match_id FROM MatchID", engine)

    # Fetch match details asynchronously with progress bar
    match_ids = match_df['match_id'].tolist()
    match_details = asyncio.run(fetch_all_match_details(tqdm(match_ids, desc="Retrieving match details")))

    # Concatenate all DataFrames
    if match_details:
        df_matchdetails = pd.concat(match_details, ignore_index=True)
        df_matchdetails.to_sql('Detalhes_Liga', con=engine, if_exists='replace', index=False)
        print("Match details saved successfully.")
    else:
        print("No match details retrieved.")

if __name__ == "__main__":
    main()
